In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from datetime import datetime
from tensorflow.keras.models import Model, load_model
from tensorflow import keras
import keras as K
from keras.layers.core import Dense
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
import matplotlib.pyplot as plt
from sklearn.model_selection import KFold
from tensorflow.keras.callbacks import ModelCheckpoint
from keras.losses import mean_squared_error
from keras import backend
from tensorflow.keras.layers import Bidirectional
import matplotlib.pyplot as plt
from matplotlib.pyplot import MultipleLocator
import time
start=time.perf_counter()

In [ ]:
trainenable = 1
data = pd.read_csv("/project/env_lstm/Swarm_data.csv", header=None) 

In [ ]:
dens = data[8]                                                            
dens = dens.astype("float")                                             
df_feature = data.drop([ 0, 1, 2, 8, 14, 15, 16], axis=1)                         
df_feature = df_feature.astype("float")                                 
df_feature = df_feature.loc[np.isfinite(df_feature).all(axis=1).squeeze(),:]
dens = dens[np.isfinite(df_feature).all(axis=1).squeeze()]
df_feature = df_feature.loc[np.isfinite(dens).squeeze(),:]
dens = dens[np.isfinite(dens).squeeze()]
df_feature = df_feature.reset_index(drop=True)
df_feature.shape

In [ ]:
idx = np.array(df_feature.index)                                           
n_data = len(df_feature)                                                   
train_ind = np.random.permutation(idx[:int(n_data * .8)])                   
valid_ind = np.random.permutation(idx[int(n_data * .8):int(n_data * .9)])    
test_ind = idx[int(n_data * .9):int(n_data * 1)]
len(test_ind)

In [ ]:
train_ind_ = idx[:int(n_data * .8)]                  
valid_ind_ = idx[int(n_data * .8):int(n_data * .9)]    
test_ind_ = idx[int(n_data * .9):int(n_data * 1)]
a=data[0]
print('训练数据：',a[train_ind_[0]],'-',a[train_ind_[-1]])
print('验证数据：',a[valid_ind_[0]],'-',a[valid_ind_[-1]])
print('测试数据：',a[test_ind_[0]],'-',a[test_ind_[-1]])

In [ ]:
start_time = "2013-03-17"  
end_time =  "2013-03-19"     
test_ind = data[(data[0]>=start_time) & (data[0]<=end_time)].index
print(test_ind[0],test_ind[-1],test_ind.shape)

In [ ]:
scaler_data = StandardScaler()             
scaler_dens = StandardScaler()             
train_feature = df_feature.loc[train_ind]  
scaler_data.fit(train_feature)               
feature = scaler_data.transform(df_feature)  
scaler_dens.fit(dens.values.reshape(-1, 1))   
dens = scaler_dens.transform(dens.values.reshape(-1, 1))   
feature = np.array(feature)
dens = np.reshape(dens, (-1, 1))

history_length = 5     
swarm_nums = 8        
omni2_num = 54        
ensembel = 0

In [ ]:
def models(input_dim = 224):  
#     import os
#     os.environ["CUDA_VISIBLE_DEVICES"] = "-1"

    model = K.models.Sequential()

    model.add(K.layers.Dense(units=64, input_dim=input_dim, kernel_initializer = 'normal', activation='sigmoid')) 
    model.add(K.layers.Dropout(0.2))
    model.add(K.layers.BatchNormalization())
    model.add(K.layers.Dense(units=32, kernel_initializer = 'normal', activation='sigmoid'))
    model.add(K.layers.Dropout(0.2))
    model.add(K.layers.BatchNormalization())
    model.add(K.layers.Dense(units=16, kernel_initializer = 'normal', activation='sigmoid'))
    model.add(K.layers.Dropout(0.2))
    model.add(K.layers.BatchNormalization())
    model.add(K.layers.Dense(units=16, kernel_initializer = 'normal', activation='sigmoid'))
    model.add(K.layers.Dropout(0.2))
    model.add(K.layers.BatchNormalization())
    model.add(K.layers.Dense(units=1))
    model.summary()

    model.compile(loss=rmse, optimizer='Adamax' ,metrics=['mse'])   
    return model

In [ ]:
def generate_arrays_from_file(train_index):    
    while 1:    
        for j in range(batch_step_nums):             
            cur_idx = train_index[j*batch_size: (j+1)*batch_size] 
            cur_feature = np.concatenate([feature[cur_idx - k, 8:] for k in range(1, history_length)], axis=-1) 
            cur_feature = np.concatenate([cur_feature, feature[cur_idx, :8]], axis=-1)
            cur_dens = dens[cur_idx]              
            yield (cur_feature, cur_dens)

In [ ]:
length = len(train_feature) // 10 
parts = []
for i in range(10):
    parts.append(train_feature.index[i * length: length + i * length])
    if i == 9:
        parts[-1] = train_feature.index[i * length: ]

In [ ]:
kf = KFold(10, shuffle=False, random_state=None)
A = []
B = []
Q = []

for train_idx in parts:
    train_index, test_index = train_idx[:int(len(train_idx) * 0.9)], train_idx[int(len(train_idx) * 0.9):]
    Q.append(test_index)

    def rmse(y_true, y_pre):
        return backend.sqrt(mean_squared_error(y_true, y_pre))
    
    model = models()
    model.compile(loss=rmse,optimizer='Adamax', metrics=['mse'])  

    filepath = f'model_ensembel_{ensembel}.h5' 
    checkpoint = ModelCheckpoint(filepath,  monitor='val_mse', verbose=0, save_best_only=True, mode='min', period=1)
    callbacks_list = [checkpoint]

    batch_size = 128
    batch_step_nums = len(train_index) // batch_size

    valid_ind = np.random.permutation(idx[int(n_data * .8):int(n_data * .9)])
    valid_feature = np.concatenate([feature[valid_ind - k, 8:] for k in range(1, history_length)], axis=-1)
    valid_feature = np.concatenate([valid_feature, feature[valid_ind, :8]], axis=-1) 
    valid_dens = dens[valid_ind]
    if trainenable:
        history=model.fit_generator(generate_arrays_from_file(train_index),
                            steps_per_epoch=batch_step_nums,
                            validation_data=(valid_feature, valid_dens),
                            epochs=20,
                            max_queue_size=1,
                            workers=1,
                            callbacks=callbacks_list)
    test_feature = np.concatenate([feature[test_index - k, 8:] for k in range(1, history_length)], axis=-1)

    test_feature = np.concatenate([test_feature, feature[test_index, :8]], axis=-1) 
    test_dens = dens[test_index]   
    model = load_model(f'model_ensembel_{ensembel}.h5',custom_objects={'rmse': rmse}) 
    pr = model.predict(test_feature)
    A.append(pr)
    test_feature = np.concatenate([feature[test_ind - k, 8:] for k in range(1, history_length)], axis=-1)
    test_feature = np.concatenate([test_feature, feature[test_ind, :8]], axis=-1)
    pr = model.predict(test_feature)
    B.append(pr)

    ensembel += 1


In [ ]:
AC = np.vstack(A)
BC = np.hstack(B)
BC = BC.mean(axis=1)
QI = np.hstack(Q)
print(AC.shape)
print(BC.shape)
print(QI.shape)

In [ ]:
model = keras.models.Sequential([tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(20, return_sequences=True, input_shape=[None, 1])),
                                 tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(10)),
                                 keras.layers.Dense(1)])

def rmse(y_true, y_pre):
    return backend.sqrt(mean_squared_error(y_true, y_pre))

model.compile(loss=rmse,
              optimizer='Adamax', metrics=['mse'])

filepath = 'model_ensembel_final.h5'
checkpoint = ModelCheckpoint(filepath,  monitor='val_mse', verbose=1, save_best_only=True, mode='min', period=1)
callbacks_list = [checkpoint]
if trainenable:
    history1=model.fit(AC.reshape(-1, 1, 1),dens[QI],
              epochs=20,
              batch_size = 128,
              verbose=1,
              validation_split = 0.1,
              callbacks=callbacks_list)
    # model.fit():将训练数据在模型中训练一定次数，返回loss和测量指标
end=time.clock()
use_time=end-start
print('train time:',use_time)

In [ ]:
plt.rcParams['figure.figsize']=(6, 4) # 图像尺寸大小
loss1=history1.history["loss"]
test_loss1 = history1.history["val_loss"]
plt.plot(loss1, label='train', color='r')
plt.plot(test_loss1, label='test',color='b')
x_major_locator=MultipleLocator(2) 
ax=plt.gca()
ax.xaxis.set_major_locator(x_major_locator)
plt.xlim(-0.5,20)
plt.yticks(fontproperties='Times New Roman', size=10)
plt.xticks(fontproperties='Times New Roman', size=10)
plt.xlabel('epoch', fontsize=15, labelpad=1)
plt.ylabel('loss', fontsize=15, labelpad=1)
plt.tick_params(direction='in', width=1, length=2, pad=3)
plt.legend(fontsize=10)
plt.savefig("loss_epoch.jpg",bbox_inches='tight',dpi=300,pad_inches=0.0)
plt.show()

In [ ]:
# densitypre = scaler_dens.inverse_transform(AC.reshape(-1,1))
# densityopt = scaler_dens.inverse_transform(dens[QI].reshape(-1,1))
densitypre = scaler_dens.inverse_transform(BC.reshape(-1,1))
densityopt = scaler_dens.inverse_transform(dens[test_ind].reshape(-1,1))
print(densitypre[0:1],'\n',densityopt[:1]) 

In [ ]:
from math import sqrt
mse = np.sum(( densityopt - (densitypre)) ** 2) / len(densityopt)
rmse = sqrt(mse)
r2 = 1-mse/ np.var(densityopt)
print('rmse',rmse,'r2',r2)